In [ ]:
#!/usr/bin/env python
"""
Demo for 1D Deblurring test problem on a small scale problem
--------------------------------------------------------------------------
Created in 2023 for TRIPs-Py library
"""
__authors__ = "Mirjeta Pasha, Silvia Gazzola, Connor Sanderford, and Ugochukwu Obinna Ugwu"
__affiliations__ = 'Tufts University, University of Bath, Arizona State University, and Tufts University'
__copyright__ = "Copyright 2023, TRIPs-Py library"
__license__ = "GPL"
__version__ = "1.0"
__email__ = "mirjeta.pasha@tufts.edu; mirjeta.pasha1@gmail.com; sg968@bath.ac.uk; csanderf@asu.edu; connorsanderford@gmail.com; Ugochukwu.Ugwu@tufts.edu"

In [ ]:
import scipy.stats as sps
import scipy.io as spio
# import matplotlib
import matplotlib.pyplot as plt
import os
# import astra
from venv import create
# import pylops
# from scipy.ndimage import convolve
from trips.test_problems.Deblurring1D import *
from trips.utilities.operators import *
from trips.utilities.helpers import *
from trips.solvers.GKS import *
from trips.solvers.MMGKS import *
from trips.solvers.Tikhonov import Tikhonov
from trips.solvers.tSVD import *
from trips.solvers.Hybrid_GMRES import *
from trips.solvers.Hybrid_LSQR import *

#### In this notebook we illustrate how to use the Deblurring1D class. The main features are:
1. Define a Deblurring1D problem where the forward operator can be formed explicitly as a matrix
 - 1.0 We compute the naive solution
 - 1.1 We show how to compute the SVD and the truncated SVD solution
 - 1.2 We use regularization methods for computing an approximate solution
   - 1.2.1. Hybrid_GMRES
   - 1.2.2. MMGKS


#### 1. Define e Deblurring1D problem where the forward operator can be accessed explicitly as a matrix

###### We define an operator, choose an image of any given size and plot the true image and the blurred and noisy data as follows:

In [ ]:
Deblur1D = Deblurring1D(CommitCrime = True)
nx = 200
x_true = Deblur1D.gen_xtrue(nx, test = 'curve0')
## If you would like to create the operator only
A = Deblur1D.forward_Op_1D(parameter = 30, nx = nx) 
## The following creates the data b_true
b_true = Deblur1D.gen_data(x_true)
(d, delta) = Deblur1D.add_noise(b_true, 'Gaussian', noise_level = 0.02)
b_vec = d.reshape((-1,1))
Deblur1D.plot_data(d)
Deblur1D.plot_rec(x_true)

In [ ]:
plt.plot(x_true, label='x_true')
plt.plot(b_vec, label='data')
plt.axis('off')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=20)
plt.show()

##### 1.0. Compute the naive solution

In [ ]:
x_naive = np.linalg.solve(A.todense(), b_vec)
# Deblur1D.plot_rec(x_naive)
plt.plot(x_true, label='x_true')
plt.plot(x_naive, label='x_naive')
plt.axis('off')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=20)
plt.show()

###### Display the true image, the blurred image, the blurred and noisy image, and the naive reconstructed solution

In [ ]:
plt.plot(d)
plt.set_cmap('inferno')
plt.axis('off')

##### 1.1. Compute the SVD of the operartor $A$ and plot the singlar values and the singular vectors
###### If the parameter is 'A' then we give the full operator to the function plot_singular_values_svd. The SVD is then computed and the singlar values are plotted. If the parameter is 'S' then we have precomputed the SVD from which we have S and the function plot_singular_values_svd will only plot the singular values. The same applies for plotting the singular vectors. If the operaror is not given, we give the matrix V.

In [ ]:
# If the operator is sparse, we convert it to dense A.todense()
plot_singular_values_svd(Operator = A , parameter = 'A')
# If the operator is in the matrix form, but very large and sparse, the following commands can be used to compute the SVD
# import scipy
# [U, S, V] = scipy.sparse.linalg.svds(A, 100)
# 100 defines the number of singular values to be computed

##### 1.1. Compute the truncated SVD solution of a problem by specifying how many singular values you want to keep after truncation

In [ ]:
# truncated_value = 100
b_vec = d.reshape((-1,1))
(x_tsvd, truncated_value) = tSVD_sol(A.todense(), b_vec, regparam = 'gcv', delta = delta)
plt.plot(x_true, label='x_true')
plt.plot(x_tsvd, label='tSVD')
plt.axis('off')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=20)
plt.show()

In [ ]:
truncated_value

- 1.2 We use regularization methods for computing an approximate solution
   - 1.2.1. Hybrid_GMRES
   - 1.2.2. Hybrid_LSQR
   - 1.2.3. MMGKS

##### Illustrate the ill-posedness of some inverse problem examples and show the need for regularization
<p>In this example we illustrate the following:

Solve the regularized problem with Hybrid_GMRES, Hybrid_LSQR

$\|\mathbf A{\mathbf x} - {\mathbf b}\|^2_2 + \lambda \|\mathbf L\mathbf x\|_2^2$ with $\mathbf L$ is the discretization of the two-dimensional first derivative operator for GKS and $L = I$ for Hybrid_lsqr, for an computed value of the regularization parameter $\lambda$. 


In [ ]:
b_vec = d.reshape((-1,1))
(x_hybrid_gmres, info_hybrid_gmres) = Hybrid_GMRES(A, b_vec, n_iter = 100, regparam = 'dp', x_true = x_true, delta = delta)

Solve the regularized problem with MMGKS

$\|\mathbf A{\mathbf x} - {\mathbf b}\|_2 + \lambda \|\mathbf L\mathbf x\|_q$ with $\mathbf L$ is the discretization of the two-dimensional first derivative operator, for an optimal value of $\lambda$. The value of $q$ can be choosen from (0, 2].

In [ ]:
L = first_derivative_operator(nx)
def gsvd_tik_sol(A,L,b_vec,mu):
    U, _, Z, C, S = gsvd(A,L) 
    Y = np.linalg.inv(Z.T)
    xsol = Y@np.linalg.inv(C.T@C+mu*S.T@S)@C.T@(U.T@b_vec)
    return xsol
# mu = 0.0001
mu= 0.001
xsolgsvd1 = gsvd_tik_sol(A.todense(),L.todense(),b_vec,mu)

mu = 10
xsolgsvd2 = gsvd_tik_sol(A.todense(),L.todense(),b_vec,mu)

plt.plot(x_true, label='x_true')
# plt.plot(xsolgsvd1,label='gsvd') # this is essentially b_vec
plt.plot(xsolgsvd2,label='gsvd') # this is essentially b_vec
# plt.axis('off')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=15)
plt.show()


In [ ]:
print(L.todense)

In [ ]:
L = gen_first_derivative_operator(nx)
L = L.todense()
L = np.vstack((L, np.zeros((1,nx))))
print(L.shape)
print(L)

In [ ]:
U, _, Z, C, S = gsvd(A.todense(),L) 

In [ ]:
# def gsvd(A, B):
#     vm1, p = A.shape
#     vm2, p = B.shape
#     if not (vm1 == vm2 and vm1 >= vm2 >= p):
#         raise ValueError("Invalid input dimensions. A should be of size mxp, and B should be of size nxp with m >= n >= p.")
#     QA, A = np.linalg.qr(A,'reduced')
#     m = p
#     QB, B = np.linalg.qr(B,'reduced')
#     n = p
#     Q, R = np.linalg.qr(np.concatenate((A, B),axis=0),'reduced')
#     U, V, Z, C, S = csd(Q[:m, :], Q[m:m+n, :])
#     X = R.T @ Z
#     U = QA @ U
#     V = QB @ V
#     return U, V, X, C, S
# def csd(Q1, Q2):
#     m, p = Q1.shape
#     n = Q2.shape[0]
#     U, C, Z = np.linalg.svd(Q1)
#     C = np.diag(C)
#     Z = Z.T
#     q = min(m, p)
#     i = np.arange(q)
#     j = np.arange(q-1, -1, -1)
#     C[i, i] = C[j, j]
#     U[:, i] = U[:, j]
#     Z[:, i] = Z[:, j]
#     S = Q2 @ Z
#     k = np.max(np.concatenate(([0], 1+np.where(np.diag(C) <= 1/np.sqrt(2))[0])))
#     V, _ = np.linalg.qr(S[:,:k],'complete')
#     S = V.T @ S
#     r = min(k, m)
#     S[:, :r] = diagf(S[:, :r])
#     if k < min(n, p):
#         r = min(n, p)
#         i = np.arange(k, n)
#         j = np.arange(k, r)
#         UT, ST, VT = np.linalg.svd(S[np.ix_(i, j)])
#         ST = np.diag(ST)
#         if k > 0:
#             S[:k, j] = 0
#         S[np.ix_(i,j)] = ST
#         C[:, j] = C[:, j] @ VT.T
#         V[:, i] = V[:, i] @ UT
#         Z[:, j] = Z[:, j] @ VT.T
#         i = np.arange(k, q)
#         Q, R = np.linalg.qr(C[np.ix_(i, j)])
#         C[np.ix_(i, j)] = diagf(R)
#         U[:, i] = U[:, i] @ Q
#     print(U.shape)
#     print(C.shape)
#     print(max(0, p - m))
#     U, C = diagp(U, C, max(0, p - m))
#     C = np.real(C)
#     V, S = diagp(V, S, 0)
#     S = np.real(S)
#     return U, V, Z, C, S
# def diagk(X, k):
#     if not np.isscalar(X) and not np.isscalar(k):
#         m, n = X.shape
#         if k >= 0:
#             diag_len = min(n - k, m)
#             diag = X[:diag_len, k:k+diag_len]
#         else:
#             diag_len = min(m + k, n)
#             diag = X[-k:-k+diag_len, :diag_len]
#     else:
#         diag = np.diagonal(X, k)
#     return diag.flatten()
# def diagf(X):
#     return np.triu(np.tril(X))
# def diagp(Y, X, k):
#     D = diagk(X, k)
#     j = np.where(np.real(D) < 0) or np.where(np.imag(D) != 0)
#     j = np.asarray(j, dtype=int).flatten()
#     D = np.diag(np.conj(D[j]) / np.abs(D[j]))
#     print('np.ix_(j)')
#     print(np.ix_(j))
#     print(np.ix_(j)[0])
#     print('Y[:, np.ix_(j)[0]].shape')
#     print(Y[:, np.ix_(j)[0]].shape)
#     print('(Y[:, np.ix_(j)] @ D.T).shape')
#     print((Y[:, np.ix_(j)] @ D.T).shape)
#     print('(np.matmul(Y[:, np.ix_(j)], D.T)).shape')
#     print((np.matmul(Y[:, np.ix_(j)], D.T)).shape)
#     print('(D.T).shape')
#     print((D.T).shape)
#     Y[:, np.ix_(j)[0]] = np.matmul(Y[:, np.ix_(j)], D.T)
#     X[j, :] = D @ X[j, :]
#     X = X + 0  # Use "+0" to set possible -0 elements to 0
#     return Y, X

In [ ]:
def gsvd_tik_sol(A,L,b_vec,mu):
    U, _, Z, C, S = gsvd(A,L) 
    Y = np.linalg.inv(Z.T)
    xsol = Y@np.linalg.inv(C.T@C+mu*S.T@S)@C.T@(U.T@b_vec)
    return xsol

mu= 0.001
xsolgsvd1 = gsvd_tik_sol(A.todense(),L,b_vec,mu)

In [ ]:
from trips.solvers.Tikhonov import Tikhonov
# L = np.eye(int(A.shape[1]))
# L = first_derivative_operator(nx)
L = gen_first_derivative_operator(nx)
L = L.todense()
L = np.vstack((L, np.zeros((1,nx))))
# print(L)
x_Tikh, lambda_Tikh = Tikhonov(A, b_vec, L, x_true, regparam = 0.02)
plt.plot(x_Tikh)
xsolgsvd1 = gsvd_tik_sol(A.todense(),L,b_vec,0.02)
plt.plot(xsolgsvd1)

In [ ]:
L = first_derivative_operator(nx)
plt.plot(L@x_true)

In [ ]:
L = first_derivative_operator(6)
xx = np.ones((6,1))
print(xx.shape)
yy = L@xx
print(yy)

In [ ]:
plt.plot(x_Tikh)

In [ ]:
from trips.solvers.tGSVD import *

In [ ]:
from trips.utilities.decompositions import gsvd
from numpy import linalg as LA
U, V, X, C, S = gsvd(A.todense(), L.todense())
print(U.shape)
print(X.shape)
print(C.shape)
print(LA.norm(A.todense() - U@C@X.T))

In [ ]:
def tgsvd_sol(A,L,b_vec,k):
    U, _, Z, C, S = gsvd(A,L) 
    Y = np.linalg.inv(Z.T)
    xsol = Y[:,-k:]@np.linalg.inv(C[-k:,-k:])@(U[:,-k:].T@b_vec)
    return xsol
xsolgsvd = tgsvd_sol(A.todense(),L,b_vec,8)

In [ ]:
plt.plot(xsolgsvd)

In [ ]:
U, _, Z, C, S = gsvd(A.todense(),L) 
Y = np.linalg.inv(Z.T)
Y = np.linalg.inv(Z.T)

In [ ]:
plt.plot(Y[:,190])

In [ ]:
plt.plot(x_true, label='x_true')
plt.plot(xsolgsvd,label='tGSVD')

In [ ]:
# for i in range(30):
(xtGSVD, k) = tGSVD_sol(A.todense(), L.todense(), b_vec, regparam = 10)
plt.plot(x_true)
plt.plot(xtGSVD)

In [ ]:
(xtGSVD, k) = tGSVD_sol(A.todense(), L.todense(), b_vec, regparam = 'gcv', delta = delta)

In [ ]:
plt.plot(x_true, label='x_true')
plt.plot(xtGSVD,label='tGSVD')
plt.axis('off')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=20)
plt.show()

In [ ]:
# Compute a solution by GKS
# Define a derivative operator as a regularization operator
L = first_derivative_operator(nx)
data_vec = d.reshape((-1,1))
# Use GKS to compute an approximate solution
(x_mmgks, info_mmgks) = MMGKS(A, data_vec, L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 100, regparam = 'gcv', x_true = x_true.reshape(-1,1), delta = delta)

In [ ]:
print(info_mmgks)

In [ ]:
plt.semilogy(info_mmgks['relError'])
# plt.plot(info_hybrid_lsqr['relError'])
plt.semilogy(info_hybrid_gmres['relError'])

In [ ]:
plt.semilogy(info_mmgks['regParam_history'])
# plt.plot(info_hybrid_lsqr['relError'])
plt.semilogy(info_hybrid_gmres['regParam_history'])

In [ ]:
plot_x_true = plt.plot(x_true, label = 'x_true')
plot_tsvd = plt.plot(d, label = 'data')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=15)
plt.axis('off')
plt.show()

In [ ]:
plot_x_true = plt.plot(x_true, label = 'x_true')
plot_tsvd = plt.plot(x_tsvd, label = 'x_tsvd')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=15)
plt.axis('off')
plt.show()

In [ ]:
plot_x_true = plt.plot(x_true, label = 'x_true')
plot_hybrid_gmres = plt.plot(x_hybrid_gmres, label = 'Hybrid_GMRES')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=15)
plt.axis('off')
plt.show()

In [ ]:
plot_tsvd = plt.plot(x_true, label='x_true')
plot_mmgks = plt.plot(x_mmgks, label='MMGKS')
plot_mmgks = plt.plot(info_mmgks['xHistory'][20])
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=15)
plt.axis('off')
plt.show()

In [ ]:
print(info_mmgks['xHistory'][2])

In [ ]:
plot_tsvd = plt.plot(x_true, label='x_true')
plot_mmgks = plt.plot(x_naive, label='x_naive')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize=15)
plt.axis('off')
plt.show()

In [ ]:
from trips.solvers.Tikhonov import Tikhonov
L = np.eye(int(A.shape[1]))
x_Tikh, lambda_Tikh = Tikhonov(A, b_vec, L, x_true, regparam = 'gcv')

In [ ]:
plt.plot(x_true, label='x_true')
plt.plot(x_Tikh, label='Tikhonov')